In [ ]:
# Note: make sure to execute `python3 run.py` before running this example 
import os
import numpy as np
import rasterio

# Define the folder path
input_folder_path = 'data/Pairs_MODIS_Landsat_filled'

# Define the unique dates you want to loop through
# This example assumes you have the unique dates in 'formatted_dates' in the format 'YYYY-MM-DD'
formatted_dates =  np.load('data/commun_dates.npy')

# Initialize empty lists to hold the images
landsat_images = []
modis_images = []

# Loop over each date
for date in formatted_dates:
    # Format the date for filename (YYYYMMDD)
    date_for_filename = date.replace('-', '')

    # Construct filenames for Landsat and MODIS
    landsat_filename = f"L_{date_for_filename}.tif"
    modis_filename = f"M_{date_for_filename}.tif"

    # Load Landsat image
    landsat_image_path = os.path.join(input_folder_path, landsat_filename)
    if os.path.exists(landsat_image_path):
        with rasterio.open(landsat_image_path) as src:
            landsat_images.append(src.read(1))  # Read the first band and append to the list

    # Load MODIS image
    modis_image_path = os.path.join(input_folder_path, modis_filename)
    if os.path.exists(modis_image_path):
        with rasterio.open(modis_image_path) as src:
            modis_images.append(src.read(1))  # Read the first band and append to the list

# Convert lists to NumPy arrays
landsat_array = np.array(landsat_images)
modis_array = np.array(modis_images)

# Optional: Print the shapes of the loaded arrays
print("Landsat images shape:", landsat_array.shape)
print("MODIS images shape:", modis_array.shape)


Landsat images shape: (51, 950, 950)
MODIS images shape: (51, 950, 950)


In [19]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import gridspec
from matplotlib import cm

# Assume landsat_array, modis_array, and formatted_dates are already defined and populated

# Create a PDF file to save the visualizations
pdf_filename = "Landsat_MODIS_Visualizations_orleans_square.pdf"

region = rasterio.open('roi/orleans_metropole.tif').read(1)

with PdfPages(pdf_filename) as pdf:

    # Loop over the number of images (assuming they are the same length)
    for i in range(len(formatted_dates)):
        # Get the current date
        date = formatted_dates[i]

        # Get the Landsat and MODIS images
        landsat_image = landsat_array[i]
        modis_image = modis_array[i]

        region_mask = region > 0
        alpha = np.where(region_mask, 1.0, 0.6)
        
        # Calculate min and max values for normalization
        overall_min = min(np.min(landsat_image), np.min(modis_image))
        overall_max = max(np.max(landsat_image), np.max(modis_image))

        # Create a grid with two image plots and one for the color bar
        fig = plt.figure(figsize=(12, 6))
        gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.05)

        # Visualization parameters for MODIS
        visualization_LST_modis = {
            'min': overall_min,
            'max': overall_max,
            'palette': ['040274', '040281', '0502a3', '0502b8', '0502ce',
                        '0502e6', '0602ff', '235cb1', '307ef3', '269db1',
                        '30c8e2', '32d3ef', '3be285', '3ff38f', '86e26f',
                        '3ae237', 'b5e22e', 'd6e21f', 'fff705', 'ffd611',
                        'ffb613', 'ff8b13', 'ff6e08', 'ff500d', 'ff0000',
                        'de0101', 'c21301', 'a71001', '911003']
        }

        # Normalize the images
        norm = mcolors.Normalize(vmin=visualization_LST_modis['min'], vmax=visualization_LST_modis['max'])
        cmap = mcolors.ListedColormap(['#' + color for color in visualization_LST_modis['palette']])

        landsat_image_rgb = cm.ScalarMappable(cmap=cmap, norm=norm).to_rgba(landsat_image)
        landsat_image_rgb[..., 3] = alpha
        
        modis_image_rgb = cm.ScalarMappable(cmap=cmap, norm=norm).to_rgba(modis_image)
        modis_image_rgb[..., 3] = alpha
        
        # Plot Landsat image in the first subplot
        ax0 = plt.subplot(gs[0])
        ax0.imshow(landsat_image_rgb, cmap=cmap, norm=norm)
        ax0.axis('off')  # Hide axes
        ax0.contour(region_mask, levels=[0.5], colors='black', linewidths=1)

        # Plot MODIS image in the second subplot
        ax1 = plt.subplot(gs[1])
        im = ax1.imshow(modis_image_rgb, cmap=cmap, norm=norm)
        ax1.axis('off')  # Hide axes
        ax1.contour(region_mask, levels=[0.5], colors='black', linewidths=1)

        # Add color bar in the third subplot (separate axis)
        cbar_ax = plt.subplot(gs[2])
        cbar = fig.colorbar(im, cax=cbar_ax)
        cbar.set_label('Temperature (°C)', fontsize=12)

        # Add date label in the middle
        fig.suptitle(date, fontsize=16, y=0.95)

        # Save the current figure to the PDF
        pdf.savefig(fig)
        plt.close(fig)

print(f"PDF file '{pdf_filename}' created successfully!")


PDF file 'Landsat_MODIS_Visualizations_orleans_square.pdf' created successfully!
